In [12]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [13]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

228813984/228813984 [==============================] - 195s 1us/step


In [14]:
#making windows path
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos')

In [15]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [16]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [17]:
#making labels
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [18]:
#resizing images
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [19]:
X = np.array(X)
y = np.array(y)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [22]:
#scaling images
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [24]:
#data augmentation to improve accuracy
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(180, 
                                                              180,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [25]:
#building cnn
num_classes = 5

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
model.fit(X_train_scaled, y_train, epochs=30)  

Epoch 1/30
86/86 [==============================] - 78s 838ms/step - loss: 1.4968 - accuracy: 0.3536
Epoch 2/30
86/86 [==============================] - 69s 800ms/step - loss: 1.1433 - accuracy: 0.5411
Epoch 3/30
86/86 [==============================] - 70s 815ms/step - loss: 1.0104 - accuracy: 0.5952
Epoch 4/30
86/86 [==============================] - 70s 812ms/step - loss: 0.9077 - accuracy: 0.6363
Epoch 5/30
86/86 [==============================] - 69s 806ms/step - loss: 0.8281 - accuracy: 0.6795
Epoch 6/30
86/86 [==============================] - 66s 766ms/step - loss: 0.7816 - accuracy: 0.6966
Epoch 7/30
86/86 [==============================] - 67s 776ms/step - loss: 0.7531 - accuracy: 0.7122
Epoch 8/30
86/86 [==============================] - 68s 786ms/step - loss: 0.6761 - accuracy: 0.7493
Epoch 9/30
86/86 [==============================] - 66s 767ms/step - loss: 0.6379 - accuracy: 0.7584
Epoch 10/30
86/86 [==============================] - 70s 808ms/step - loss: 0.6068 - accura

In [26]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 4s 120ms/step - loss: 1.0960 - accuracy: 0.7342


[1.096006155014038, 0.7342047691345215]